!pip3 install openai
!pip3 install gradio

In [ ]:
import os
import json
from typing import List, Dict, Tuple

import openai
import gradio as gr

In [ ]:
# TODO: 设置你的 OPENAI API 密钥，这里以阿里云 DashScope API为例进行演示
OPENAI_API_KEY = "sk-tqeffovhnxuhojxzzbitlzhgpfguaqredvigjzjjybbumvsf"
client = openai.OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.siliconflow.cn/v1",  # 使用阿里云大模型API
)

# 检查是否正确设置了 API
# 如果一切正常，你将看到 "API 设置成功！！"
try:
    response = client.chat.completions.create(
            model="deepseek-ai/DeepSeek-R1",  # 使用阿里云 DashScope 的模型
            messages=[{'role': 'user', 'content': "测试"}],  # 设置一个简单的测试消息
            max_tokens=1,
    )
    print("API 设置成功！！")  # 输出成功信息
except Exception as e:
    print(f"API 可能有问题，请检查：{e}")  # 输出详细的错误信息

In [ ]:
# TODO: 在此处输入用于摘要的提示词
prompt_for_summarization = "请为以下文章生成一个简洁的摘要，要求如下：提取核心观点和关键信息；保持逻辑清晰，语言简练；避免添加个人观点或额外解释；如果文章涉及数据或研究结果，请保留重要数据;生成的内容请保持完整"

# 重置对话的函数
def reset() -> List:
    return []

# 调用模型生成摘要的函数
def interact_summarization(prompt: str, article: str, temp=1.0) -> List[Tuple[str, str]]:
    '''
    * 参数:
      - prompt: 我们在此部分中使用的提示词
      - article: 需要摘要的文章
      - temp: 模型的温度参数。温度用于控制聊天机器人的输出。温度越高，响应越具创造性。
    '''
    input = f"{prompt}\n{article}"
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1",  # 使用阿里云 DashScope 的模型
        messages=[{'role': 'user', 'content': input}],
        temperature=temp,
        max_tokens=200,  # 你需要注意到这里设置了文本的长度上限。
    )

    return [(input, response.choices[0].message.content)]

# 导出整个对话内容的函数
def export_summarization(chatbot: List[Tuple[str, str]], article: str) -> None:
    '''
    * 参数:
      - chatbot: 模型的对话记录，存储在元组列表中
      - article: 需要摘要的文章
    '''
    target = {"chatbot": chatbot, "article": article}
    with open("files/part1.json", "w") as file:
        json.dump(target, file)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown("# 第1部分：摘要\n填写任何你喜欢的文章，让聊天机器人为你总结！")
    chatbot = gr.Chatbot()
    prompt_textbox = gr.Textbox(label="提示词", value=prompt_for_summarization, visible=False)
    article_textbox = gr.Textbox(label="文章", interactive=True, value="Transformer是谷歌研究人员开发的一种深度学习架构，基于2017 年论文《Attention Is All You Need》中提出的多头注意力机制。[ 1 ]文本被转换为称为token的数值表示，每个 token 通过从词嵌入表中查找转换为向量。[ 1 ]在每一层，每个token都通过并行多头注意力机制在上下文窗口范围内与其他（未屏蔽的）token进行语境化，从而放大关键token的信号，减弱不太重要的 token。Transformer 的优势在于没有循环单元，因此所需的训练时间比早期的循环神经架构(RNN)（例如长短期记忆(LSTM)）要少。[ 2 ]后来的变体已被广泛用于在大型（语言）数据集上训练大型语言模型(LLM) 。[ 3 ]Transformer 最初是作为对以前机器翻译 架构的改进而开发的，[ 4 ] [ 5 ]但此后已发现了许多应用。它们用于大规模自然语言处理、计算机视觉（视觉变换器）、强化学习、[ 6 ] [ 7 ] 音频、[ 8 ] 多模态学习、机器人技术、[ 9 ]甚至下棋。[ 10 ]它还导致了预训练系统的发展，例如生成式预训练 Transformer（GPT）[ 11 ]和BERT [ 12 ]（来自 Transformer 的双向编码器表示）。")

    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")

    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")

    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")

    # 连接按钮与函数
    sent_button.click(interact_summarization, inputs=[prompt_textbox, article_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_summarization, inputs=[chatbot, article_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

读取内容, 生成摘要

In [ ]:
# 加载对话记录的 JSON 文件
with open("files/part1.json", "r") as f:
    context = json.load(f)

chatbot = context['chatbot']  # 获取对话记录
article = context['article']  # 获取原始文章
summarization = chatbot[0][-1]  # 获取摘要结果

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown("# 第1部分：摘要\n你可以查看文章和摘要！")
    chatbot = gr.Chatbot(value=context['chatbot'])  # 加载对话历史
    article_textbox = gr.Textbox(label="文章", interactive=False, value=context['article'])  # 显示原始文章

    # 构建展示摘要和原文的部分
    with gr.Column():
        gr.Markdown("# 只是一个检查")
        gr.Textbox(label="文章", value=article, show_copy_button=True)  # 显示并允许复制原文
        gr.Textbox(label="摘要", value=summarization, show_copy_button=True)  # 显示并允许复制摘要

# 启动 Gradio 界面
demo.launch(debug=True)

# 最大的感受deepseek回复的内容似乎有字数的限制，导致生成的摘要不够完整
